In [17]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import image_uris 
from sagemaker.session import s3_input, Session
import warnings
warnings.filterwarnings('ignore')

In [18]:
bucket_name="myproject--1"
bucket_region=boto3.session.Session().region_name
bucket_region

'ap-south-1'

In [19]:
s3=boto3.resource('s3')
try:
    if bucket_region=="ap-south-1":
        s3.create_bucket(Bucket=bucket_name,CreateBucketConfiguration={'LocationConstraint': bucket_region})
    print("s3 bucket created successfully")
except Exception as e:
    print("s3 error :",e)
    

s3 error : An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [20]:
prefix="xgboost-as-a-built-in-algo"
output_path=f"s3://{bucket_name}/{prefix}/output"
print(output_path)

s3://myproject--1/xgboost-as-a-built-in-algo/output


#### Downloading Data and Storing it in s3

In [21]:
import pandas as pd
import urllib

try:
    urllib.request.urlretrieve("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv","data_clean.csv")
    print("Dataset downloaded successfully as data_clean.csv")
    model_data=pd.read_csv("./data_clean.csv",index_col=0)
    
except Exception as e:
    print("Data Load error",e)


    

Dataset downloaded successfully as data_clean.csv


In [22]:
model_data.head()

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,56,1,999,0,1,0,0,0,0,1,...,0,1,0,0,0,0,1,0,1,0
1,57,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
2,37,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
3,40,1,999,0,1,0,1,0,0,0,...,0,1,0,0,0,0,1,0,1,0
4,56,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0


In [44]:
## train test split
import numpy as np
train_data, test_data=np.split(model_data.sample(frac=1, random_state=101),[int(0.7*len(model_data))])
print(train_data.shape,test_data.shape)

(28831, 61) (12357, 61)


In [24]:
test_data

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
28737,33,1,999,0,1,0,0,1,0,0,...,0,0,1,0,0,0,1,0,1,0
19323,46,2,999,0,1,0,0,1,0,0,...,0,0,0,0,1,0,1,0,0,1
34636,36,2,999,1,1,0,0,1,0,0,...,0,0,1,0,0,1,0,0,1,0
1651,31,1,999,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,1,0
23543,37,12,999,0,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12363,30,2,999,0,1,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,0
5695,50,2,999,0,1,0,0,0,1,0,...,0,1,0,0,0,0,1,0,1,0
8006,35,2,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
17745,44,21,999,0,1,0,0,1,0,0,...,0,0,0,1,0,0,1,0,1,0


In [25]:
### Saving Train And Test Into Buckets
## In sagemaker, dataset format is to keep target column as first column in dataset
import os
pd.concat([train_data['y_yes'],train_data.drop(['y_no','y_yes'], axis=1)],axis=1).to_csv('train.csv',index=False,header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

In [26]:
s3_input_train = sagemaker.TrainingInput(s3_data=f"s3://{bucket_name}/{prefix}/train", content_type='csv')
s3_input_train

In [27]:
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

#### Building and Training Model

In [28]:
container=image_uris.get_training_image_uri(boto3.Session().region_name,
                       framework=' ',
                       framework_version='1.7-1'
                       )

In [29]:
container

'720646828776.dkr.ecr.ap-south-1.amazonaws.com/sagemaker-xgboost:1.7-1'

In [30]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [32]:
 ##construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          use_spot_instances=True,
                                          max_run=300,
                                          max_wait=600)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [33]:
estimator.fit({'train':s3_input_train,'validation':s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-12-03-18-21-19-840


2023-12-03 18:21:19 Starting - Starting the training job...
2023-12-03 18:21:33 Starting - Preparing the instances for training......
2023-12-03 18:22:47 Downloading - Downloading input data...
2023-12-03 18:23:07 Training - Downloading the training image......
2023-12-03 18:24:18 Uploading - Uploading generated training model[2023-12-03 18:24:13.695 ip-10-0-102-245.ap-south-1.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-12-03 18:24:13.717 ip-10-0-102-245.ap-south-1.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2023-12-03:18:24:14:INFO] Imported framework sagemaker_xgboost_container.training
[2023-12-03:18:24:14:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2023-12-03:18:24:14:INFO] No GPUs detected (normal if no gpus installed)
[2023-12-03:18:24:14:INFO] Running XGBoost Sagemaker in algorithm mode
[2023-12-03:18:24:14:INFO] Determined 0 GPU(s) available o

#### Deploy ML model

In [34]:
xgb_predictor=estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2023-12-03-18-35-42-071
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2023-12-03-18-35-42-071
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2023-12-03-18-35-42-071


------!

#### Predicting test data

In [70]:
from sagemaker.serializers import CSVSerializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = CSVSerializer()
predictions=xgb_predictor.predict(test_data_array).decode('utf-8')
predictions_array = np.fromstring(predictions[1:], sep='\n') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


In [69]:
predictions_array

array([0.10204415, 0.07383633, 0.06221998, ..., 0.03871584, 0.01569422,
       0.07269247])

In [71]:
predictions_array

array([0.10204415, 0.07383633, 0.06221998, ..., 0.03871584, 0.01569422,
       0.07269247])

In [73]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.5%

Predicted      No Purchase    Purchase
Observed
No Purchase    90% (10763)    35% (161)
Purchase        10% (1133)     65% (300) 



#### Deleting endpoints

In [74]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2023-12-03-18-35-42-071


[{'ResponseMetadata': {'RequestId': 'JVDC7HZ3SDSZ2BTV',
   'HostId': 'IDoD1XFkkIGLhHA/ACsT9FSDZy4tNYcW9ETs7TcmILmkoAGNegj5MhpkN7qOBNhOwj5TbPJG8WU=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'IDoD1XFkkIGLhHA/ACsT9FSDZy4tNYcW9ETs7TcmILmkoAGNegj5MhpkN7qOBNhOwj5TbPJG8WU=',
    'x-amz-request-id': 'JVDC7HZ3SDSZ2BTV',
    'date': 'Sun, 03 Dec 2023 19:23:06 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2023-12-03-18-21-19-840/debug-output/events/000000000020/000000000020_worker_0.tfevents'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2023-12-03-18-21-19-840/output/model.tar.gz'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2023-12-03-18-21-19-840/debug-output/index/000000000/000000000020_worker_0.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/